In [25]:
import streamlit as st
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import os
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as plot

In [26]:
def read_feather(source_file):
    feather_file_list = os.listdir(source_file)
    df = pd.DataFrame()
    for feather_files in feather_file_list:
        if feather_files.endswith(".feather"):
            strfile = source_file + feather_files
            df1 = pd.read_feather(strfile)
            df = pd.concat([df, df1], ignore_index=True, sort=False)
    return df

In [27]:
df = read_feather("Data/Tonghop/")
df.head(2)

,level_0,index,Date,Mã sản phẩm,Mã siêu thị,Số lượng hủy NCC,Số lượng nhập,Số lượng thực hủy,Doanh thu,Số lượng bán,Tên ST,RSM tháng 6,AM tháng 6,Miền,Tỉnh/thành,Quận/huyện,Tồn kho siêu thị,Trạng thái kinh doanh,Tên sản phẩm,Nhóm hàng
0,0,0,2023-06-01,1232844000391,10005,0.0,0.0,0.000,23868.0,1.326,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,1.000,KD bình thường,THƠM MẬT NGUYÊN VỎ,Trái cây nội CL
1,1,1,2023-06-01,1232854000015,10005,0.0,0.0,-5.692,477306.0,9.292,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,10.156,KD bình thường,TÁO GALA MINI NHẬP KHẨU,Trái cây ngoại CL


In [28]:
app = JupyterDash(__name__)
product_list = df['Tên sản phẩm'].unique().tolist()
sub_group_list = df['Nhóm hàng'].unique().tolist()
store_list = df['Mã siêu thị'].unique().tolist()
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
            sub_group_list,
            sub_group_list[0],
            id='sub_group_selected'
            )],style={'width': '10%','float': 'left', 'display': 'inline'}),
        html.Div([
            dcc.Dropdown(
            options=product_list,
            value=product_list[0],
            id='product_selected'
            )],style={'width': '10%','float' : 'center', 'display': 'inline'}),
        html.Div([
            dcc.Dropdown(
            options=store_list,
            value = store_list[0],
            id='store_selected'
        )],style={'width': '10%','float' : 'right', 'display': 'inline'}),
    ]),

    dcc.Graph(id='graph-with-slider'),

])

@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('store_selected', 'value'),
    Input('sub_group_selected', 'value'),
    Input('product_selected', 'value'))
def update_figure(store_selected,sub_group_selected,product_selected):

    fig = make_subplots(rows=1, cols=2)

    data_sale = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
    data_nhap = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()
    if len(data_sale) == 0:
        data_sale = pd.pivot_table(df,index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
    if len(data_nhap) == 0:
        data_nhap = pd.pivot_table(df,index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()

    fig.add_trace(go.Scatter(x = data_nhap["Date"], y = data_nhap["Số lượng nhập"], line_shape='linear', name='Số lượng nhập'),row=1, col=1)
    fig.add_trace(go.Scatter(x = data_sale["Date"], y = data_sale["Số lượng bán"], line_shape='linear', name='Số lượng bán'),row=1, col=2)
  
    return fig


if __name__ == '__main__':
    app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/

